In [ ]:
import cv2 #importing computer vision library
import numpy as np #import numpy to convert image to matrix
import smtplib #importing smtplib to send email
import playsound #importing playsound to play the sound of alarm
Alarm_Status = False
Fire_Reported = 0

#function to play alarm if fire is detected
def play_audio():
    playsound.playsound('Alarm Sound.mp3',True)

#function to send email to fire station after detecting fire
def send_mail_function():

    recipientEmail = "Fire_Station_Email" #provide the email of fire station to take quick action
    recipientEmail = recipientEmail.lower()

    try:
        server = smtplib.SMTP('smtp.gmail.com',587 )
        server.ehlo()
        server.starttls()
        server.login("system(email)", 'Password') #system email from which email is sent to fire station
        server.sendmail('system(email)', recipientEmail, "Warning A Fire Accident has been reported")#message that is to be sent
        print("sent to {}".format(recipientEmail))
        server.close()
    except Exception as e:
        print(e)

video = cv2.VideoCapture("fire-video.mp4") # If you want to use webcam use Index like 0,1. I have used a fire video
 
while True:
    ret, frame = video.read()
    frame = cv2.resize(frame, (1000,600)) #defining and resizing the frame of the video
 
    blur = cv2.GaussianBlur(frame, (15,15), 0) #to blur the other part of the video except fire 
    hsv = cv2.cvtColor(blur, cv2.COLOR_BGR2HSV) #changing it into HSV format to identify fire colour
    #converting the captured image from video into matrix format
    lower = [18, 50, 50]
    upper = [35, 255, 255]
    lower = np.array(lower, dtype="uint8")
    upper = np.array(upper, dtype="uint8")

    mask = cv2.inRange(hsv, lower, upper)
 
    output = cv2.bitwise_and(frame, hsv, mask=mask)
    #performs bitwise operation- the AND of the values of each pixel of input image is the pixel value of output image
    size = cv2.countNonZero(mask)
    if int(size) > 15000: #defining threshold value. We can increase the value if we want to detect large fire.
        Fire_Reported = Fire_Reported + 1 #increasing the count if the size is greater than 15000
        if Fire_Reported >= 1: #if the counter increases even by 1. fire alarm is generated and email is sent
            if Alarm_Status == False:
                send_mail_function() #calling the mail function to first send the email
                play_audio() #calling the alarm function to play the alarm
                Alarm_Status = True
    else:
        print("Fire not detected") #if fire is not detected print the message "Fire not detected"

    if ret==False:
        break
    cv2.imshow("Output",output) #show the output on other window named "output"
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
 
cv2.destroyAllWindows()
video.release()